In [1]:
import csv
import re
import os
import sys
from io import open

In [2]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [3]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 input_ids,
                 input_mask,
                 e11_p, e12_p, e21_p, e22_p,
                 e1_mask, e2_mask,
                 segment_ids,
                 label_id):
                 
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.e11_p = e11_p
        self.e12_p = e12_p
        self.e21_p = e21_p
        self.e22_p = e22_p
        self.e1_mask = e1_mask
        self.e2_mask = e2_mask
        self.segment_ids=segment_ids
        self.label_id = label_id

In [4]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()


    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()


    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

In [5]:
class SemEvalProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return [str(i) for i in range(19)]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets.
        e.g.,: 
        2	the [E11] author [E12] of a keygen uses a [E21] disassembler [E22] to look at the raw assembly code .	6
        """
        examples = []
        for line in lines:
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[1]
            text_b = None
            label = line[2]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [37]:
def convert_examples_to_features(examples, max_seq_len,
                                 tokenizer, 
                                 cls_token='[CLS]',
                                 cls_token_segment_id=1,
                                 sep_token='[SEP]',
                                 pad_token=0,
                                 pad_token_segment_id=0,
                                 sequence_a_segment_id=0,
                                 mask_padding_with_zero=True,
                                 use_entity_indicator=True,
                                 return_tensors=True):
    """does this"""

    features = []
    for example in examples:
        
        tokens_a = tokenizer.tokenize(example.text_a)

        # Account for [CLS] and [SEP] with "- 2"
        special_tokens_count = 2
        if len(tokens_a) > max_seq_len - special_tokens_count:
            tokens_a = _truncate_seq(tokens_a, max_seq_len - special_tokens_count)

        tokens = [cls_token] + tokens_a + [sep_token]
        segment_ids = [cls_token_segment_id] + [sequence_a_segment_id] * (len(tokens)-1)

        assert len(tokens) == len(segment_ids)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # entity mask
        if use_entity_indicator:
            if "</e2>" not in tokens or "</e1>" not in tokens:  # remove this sentence because after max length truncation, the one entity boundary is broken
                continue 
            else:
                e11_p = tokens.index("<e1>")+1
                e12_p = tokens.index("</e1>")
                e21_p = tokens.index("<e2>")+1
                e22_p = tokens.index("</e2>")

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_len - len(input_ids)
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + \
            ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + \
            ([pad_token_segment_id] * padding_length)
        if use_entity_indicator:
            e1_mask = [0 for i in range(len(input_mask))]
            e2_mask = [0 for i in range(len(input_mask))]
            for i in range(e11_p, e12_p):
                e1_mask[i] = 1
            for i in range(e21_p, e22_p):
                e2_mask[i] = 1

        assert len(input_ids) == max_seq_len, f"Error in sample: {example.guid}, len(input_ids)={len(input_ids)}"
        assert len(input_mask) == max_seq_len
        assert len(segment_ids) == max_seq_len

        label_id = example.label

        features.append(
            InputFeatures(input_ids=input_ids,
                          input_mask=input_mask,
                          e11_p=e11_p,
                          e12_p=e12_p,
                          e21_p=e21_p,
                          e22_p=e22_p,
                          e1_mask=e1_mask,
                          e2_mask=e2_mask,
                          segment_ids=segment_ids,
                          label_id=label_id))
    
    if return_tensors:
        
        def gen():
            for ex in features:
                yield ({"input_ids": ex.input_ids, 
                        "attention_mask": ex.input_mask, 
                        "token_type_ids": ex.segment_ids,
                        "e1_mask": ex.e1_mask,
                        "e2_mask": ex.e2_mask},
                        ex.label_id
                        )

        dataset = tf.data.Dataset.from_generator(
            gen,
            ({"input_ids": tf.int32, 
            "attention_mask": tf.int32, 
            "token_type_ids": tf.int32,
            "e1_mask": tf.int32,
            "e2_mask": tf.int32}, 
            tf.int64),
            ({"input_ids": tf.TensorShape([None]), 
            "attention_mask": tf.TensorShape([None]),
            "token_type_ids": tf.TensorShape([None]),
            "e1_mask": tf.TensorShape([None]),
            "e2_mask": tf.TensorShape([None])}, 
            tf.TensorShape([])),
        )
        return dataset
    else:
        return features

In [18]:
def _truncate_seq(tokens_a, max_length):
    """Truncates a sequence """
    tmp = tokens_a[:max_length]
    if ("[E12]" in tmp) and ("[E22]" in tmp):
        return tmp
    else:
        e11_p = tokens_a.index("[E11]")
        e12_p = tokens_a.index("[E12]")
        e21_p = tokens_a.index("[E21]")
        e22_p = tokens_a.index("[E22]")
        start = min(e11_p, e12_p, e21_p, e22_p)
        end = max(e11_p, e12_p, e21_p, e22_p)
        if end-start > max_length:
            remaining_length = max_length - (e12_p-e11_p+1) - (e22_p-e21_p+1)  
            first_addback = math.floor(remaining_length/2)
            second_addback = remaining_length - first_addback
            if start == e11_p:
                new_tokens = tokens_a[e11_p: e12_p+1+first_addback] + tokens_a[e21_p-second_addback:e22_p+1]
            else:
                new_tokens = tokens_a[e21_p: e22_p+1+first_addback] + tokens_a[e11_p-second_addback:e12_p+1]
            return new_tokens
        else:
            new_tokens = tokens_a[start:end+1]
            remaining_length = max_length - len(new_tokens)
            if start < remaining_length:  # add sentence beginning back
                new_tokens = tokens_a[:start] + new_tokens 
                remaining_length -= start
            else:
                new_tokens = tokens_a[start-remaining_length:start] + new_tokens
                return new_tokens

            # still some room left, add sentence end back
            new_tokens = new_tokens + tokens_a[end+1:end+1+remaining_length]
            return new_tokens

In [7]:
processor = SemEvalProcessor()
ex = processor.get_train_examples('./data')

In [26]:
print(f"{ex[3].text_a}, {ex[3].label}")

The site was encircled by a high fence and one could see a <e1>crane</e1> lifting some heavy <e2>construction material</e2>., 0


In [10]:
import tensorflow as tf
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_special_tokens({'additional_special_tokens':
                            ['<e1>', '</e1>','<e2>','</e2>']})

print(tokenizer.all_special_tokens, tokenizer.all_special_ids)                        

100%|██████████| 231508/231508 [00:00<00:00, 669760.16B/s]['[MASK]', '[SEP]', '</e1>', '[PAD]', '[CLS]', '<e1>', '[UNK]', '<e2>', '</e2>'] [103, 102, 30523, 0, 101, 30522, 100, 30524, 30525]



In [30]:
import pandas as pd
data = pd.read_csv("./data/train.tsv", sep="\t")
sentences = data.iloc[:,1]
tokenized_sentences = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_sentences[2])

['the', 'site', 'was', 'encircled', 'by', 'a', 'high', 'fence', 'and', 'one', 'could', 'see', 'a', '<e1>', 'crane', '</e1>', 'lifting', 'some', 'heavy', '<e2>', 'construction', 'material', '</e2>', '.']


In [38]:
fe = convert_examples_to_features(ex,128,tokenizer)

In [47]:
for x,y in fe:
    print(x,y)


0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>, 'e2_mask': <tf.Tensor: id=30718, shape=(128,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>} tf.Tensor(11, shape=(), dtype=int64)
{'input_ids': <tf.Tensor: id=30725, shape=(128,), dtype=int32, numpy=
array([  101,  2004,  1996, 30522,  4049, 30523,  7434,  2185,  2013,
        1996, 30524,  3417, 30525,  1010,  2057,  2018,  1037,  3819,
        3193,  1997, 21554,  2050,  1998,  1996,  1062,  8004,  4143,
        2290,  2346,  2057,  2074, 27797,  3081,  2256, 20325, 12271,
        1012,   10

In [27]:
print(fe[3].input_ids)

[101, 1996, 2609, 2001, 25759, 2011, 1037, 2152, 8638, 1998, 2028, 2071, 2156, 1037, 30522, 11308, 30523, 8783, 2070, 3082, 30524, 2810, 3430, 30525, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
print(fe[3].e2_mask)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [33]:
print(fe[3].input_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
print(fe[3].segment_ids)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [35]:
print(fe[3].label_id)

0
